Description: stiches sensors and potholes files together into a new combined file using a given time window

In [15]:
import pandas as pd
pd.options.display.max_rows = 100
pd.set_option('display.precision', 10)  #for displaying timestamps

In [39]:
sensorsFilePath = 'data/Final_Route/t6-random_sensors.csv'
potholesFilePath = 'data/Final_Route/t6-random_potholes.csv'
combinedFilePath = 'data/Final_Route/t6-random_combined.csv'
sensorsDF = pd.read_csv(sensorsFilePath)
potholesDF = pd.read_csv(potholesFilePath)
window = 1.0   #pothole time window in seconds

In [40]:
sensorsDF.head()

,timestamp,latitude,longitude,speed,accelerometerX,accelerometerY,accelerometerZ,gyroX,gyroY,gyroZ
0,1.4930035626e+09,40.4475257722,-79.9441930614,0.0,0.2176666260,-0.9800415039,0.1583404541,0.0329595480,0.0487339837,0.1078993031
1,1.4930035627e+09,40.4475257722,-79.9441930614,0.0,0.0113525391,-0.9469299316,0.2120971680,0.0235993357,-0.0873756522,0.0175265294
2,1.4930035629e+09,40.4475257722,-79.9441930614,0.0,0.0236511230,-0.9569549561,0.2081146240,-0.0095865810,0.0053814496,0.0030866037
3,1.4930035631e+09,40.4475257822,-79.9441930619,0.0,0.0610809326,-0.9560852051,0.2014617920,-0.0148543136,0.0011598067,-0.0033537188
4,1.4930035633e+09,40.4475257822,-79.9441930619,0.0,0.0701599121,-0.9564666748,0.1945037842,-0.0127994185,-0.0009803096,0.0009376990


In [41]:
potholesDF

,timestamp
0,1.4930036857e+09
1,1.4930036867e+09
2,1.4930036926e+09
3,1.4930037090e+09
4,1.4930037140e+09
5,1.4930037149e+09
6,1.4930037164e+09
7,1.4930037173e+09
8,1.4930037186e+09
9,1.4930037228e+09


In [42]:
potholeLabels = []   #indicator for pothole at each timestamp
sensorIndex = 0
sensorTimestamp = sensorsDF['timestamp'][sensorIndex]
halfWindow = window/2

#Adds pothole indicators based on time and window
for index, potholeTimestamp in potholesDF['timestamp'].iteritems():
    while sensorTimestamp <= (potholeTimestamp + halfWindow):
        if (potholeTimestamp - halfWindow) <= sensorTimestamp:
            potholeLabels.append(True)
        else:
            potholeLabels.append(False)
        sensorIndex += 1
        sensorTimestamp = sensorsDF['timestamp'][sensorIndex]

#No potholes for remainder of data 
#(when sensorsDF larger than potholesDF)
remainingIndices = len(sensorsDF) - sensorIndex
remainingPotholeLabels = [False] * remainingIndices
potholeLabels.extend(remainingPotholeLabels)

In [43]:
sensorsDF['potholes'] = potholeLabels
sensorsDF.head()

,timestamp,latitude,longitude,speed,accelerometerX,accelerometerY,accelerometerZ,gyroX,gyroY,gyroZ,potholes
0,1.4930035626e+09,40.4475257722,-79.9441930614,0.0,0.2176666260,-0.9800415039,0.1583404541,0.0329595480,0.0487339837,0.1078993031,False
1,1.4930035627e+09,40.4475257722,-79.9441930614,0.0,0.0113525391,-0.9469299316,0.2120971680,0.0235993357,-0.0873756522,0.0175265294,False
2,1.4930035629e+09,40.4475257722,-79.9441930614,0.0,0.0236511230,-0.9569549561,0.2081146240,-0.0095865810,0.0053814496,0.0030866037,False
3,1.4930035631e+09,40.4475257822,-79.9441930619,0.0,0.0610809326,-0.9560852051,0.2014617920,-0.0148543136,0.0011598067,-0.0033537188,False
4,1.4930035633e+09,40.4475257822,-79.9441930619,0.0,0.0701599121,-0.9564666748,0.1945037842,-0.0127994185,-0.0009803096,0.0009376990,False


In [44]:
sensorsDF[sensorsDF['potholes'] == True]

,timestamp,latitude,longitude,speed,accelerometerX,accelerometerY,accelerometerZ,gyroX,gyroY,gyroZ,potholes
614,1.4930036853e+09,40.4532104320,-79.9475353756,9.9799995422,0.3191528320,-1.2776489258,0.1109619141,0.0286076764,0.2178721435,-0.0258584952,True
615,1.4930036855e+09,40.4532104320,-79.9475353756,9.9799995422,-0.2827911377,-0.8435974121,0.4779815674,0.0714238499,-0.0344826098,0.1308983623,True
616,1.4930036857e+09,40.4532104320,-79.9475353756,9.9799995422,-0.7182464600,-1.3249511719,0.6968536377,0.1267022857,0.2937772946,-0.3033319176,True
617,1.4930036859e+09,40.4532104320,-79.9475353756,9.9799995422,-0.1927032471,-0.8647766113,0.3640594482,0.0198575944,0.3210717663,-0.2629139207,True
618,1.4930036861e+09,40.4532104320,-79.9475353756,9.9799995422,0.0528717041,-1.0030517578,0.3737792969,-0.1635183572,0.2074003278,-0.1546779940,True
619,1.4930036863e+09,40.4531762056,-79.9476506348,10.9399995804,0.1708221436,-0.6574707031,0.3615417480,0.1304916976,-0.2859073873,0.1129601082,True
620,1.4930036865e+09,40.4531762056,-79.9476506348,10.9399995804,-0.3746490479,-1.2853546143,0.6541442871,-0.4907811741,0.6253592261,-0.6783534686,True
621,1.4930036867e+09,40.4531762056,-79.9476506348,10.9399995804,0.4201660156,-0.9645080566,0.1910705566,-0.0145075700,0.1032670007,-0.0505270901,True
622,1.4930036869e+09,40.4531762056,-79.9476506348,10.9399995804,0.0760345459,-0.9176940918,0.3105926514,-0.0381966543,0.0427868787,-0.0644743310,True
623,1.4930036871e+09,40.4531762056,-79.9476506348,10.9399995804,0.0223083496,-0.9021759033,0.3123168945,-0.1000493695,-0.0291240634,0.0576925913,True


In [45]:
sensorsDF.to_csv(combinedFilePath)